In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
!pip list

In [31]:
csv = pd.read_csv("D:\Files\Freelancing\pyfreelnce\Project_1\source\Python_Data_Science_and_Machine_Learning_project1_Dataset\dft-road-casualty-statistics-casualty-provisional-mid-year-unvalidated-2022 (1).csv")
df = csv

In [0]:
cleaned_df = csv.drop_duplicates()
cleaned_df

In [33]:
# Read the CSV data into a DataFrame


# Calculate the range of diversity for each numerical column
diversity_ranges = {}
for column in df.columns:
    if df[column].dtype in ['int64', 'float64']:  # Check if the column is numerical
        column_range = df[column].max() - df[column].min()
        diversity_ranges[column] = column_range + 1

# Print the range of diversity for each numerical column
for column, diversity_range in diversity_ranges.items():
    print(f"Range of diversity for '{column}': {diversity_range}")

Range of diversity for 'accident_year': 1
Range of diversity for 'vehicle_reference': 227
Range of diversity for 'casualty_reference': 148
Range of diversity for 'casualty_class': 3
Range of diversity for 'sex_of_casualty': 11
Range of diversity for 'age_of_casualty': 103
Range of diversity for 'age_band_of_casualty': 13
Range of diversity for 'casualty_severity': 3
Range of diversity for 'pedestrian_location': 11
Range of diversity for 'pedestrian_movement': 10
Range of diversity for 'car_passenger': 11
Range of diversity for 'bus_or_coach_passenger': 11
Range of diversity for 'pedestrian_road_maintenance_worker': 4
Range of diversity for 'casualty_type': 100
Range of diversity for 'casualty_home_area_type': 5
Range of diversity for 'casualty_imd_decile': 12
